In [1]:
%env SM_FRAMEWORK=tf.keras

# 패키지 불러오기
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import numpy as np 
import pandas as pd
import random
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import backend as keras

from glob import glob

import tensorflow_addons as tfa

from scipy import ndimage

import cv2
import cv2 as cv2
import multiprocessing
import os

env: SM_FRAMEWORK=tf.keras


### 2D_JSON 파일 압축 풀기

In [2]:
import zipfile

def extract_images_from_zip(zip_path, folder_name):
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        file_list = zip_file.namelist()
        for file_name in tqdm(file_list):
            if folder_name in file_name:
                zip_file.extract(file_name, path='./')
                # print(f"Extracted {file_name}")

In [3]:
''' # 라벨링 데이터 가져오기(Json 파일)
zip_path_list = ['./01.데이터/2.Validation/라벨링데이터/VL1.온전.zip',
                 './01.데이터/2.Validation/라벨링데이터/VL2.비온전(물리).zip',
                 './01.데이터/2.Validation/라벨링데이터/VL3.비온전(환경).zip',
                 './01.데이터/2.Validation/라벨링데이터/VL4.비온전(복합).zip'
                 ]

for zip_path in zip_path_list:
    extract_images_from_zip(zip_path, '2D_JSON') '''

" # 라벨링 데이터 가져오기(Json 파일)\nzip_path_list = ['./01.데이터/2.Validation/라벨링데이터/VL1.온전.zip',\n                 './01.데이터/2.Validation/라벨링데이터/VL2.비온전(물리).zip',\n                 './01.데이터/2.Validation/라벨링데이터/VL3.비온전(환경).zip',\n                 './01.데이터/2.Validation/라벨링데이터/VL4.비온전(복합).zip'\n                 ]\n\nfor zip_path in zip_path_list:\n    extract_images_from_zip(zip_path, '2D_JSON') "

In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split
import json

# 이미지 파일 위치한 폴더들과 어노테이션 파일이 위치한 폴더들의 경로
image_folders = ['./IMAGE_all/']

annotation_folders = ['./2D_JSON/']

### Annotation: B-box

In [5]:
#원본 이미지 파일 정규화 진행
#어노테이션 파일에서 'images' 정보 중 width, height 정보 이용

def normalize_bbox(bbox, image_width, image_height):
    x_min = bbox[0]
    y_min = bbox[1]
    width = bbox[2]
    height = bbox[3]

    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    norm_width = width / image_width
    norm_height = height / image_height

    return x_center, y_center, norm_width, norm_height

In [6]:
# (각각의 폴더에서) 이미지와 각 이미지에 대한 2D 어노테이션 파일
# YOLOv8 detection 모델에 적용할 수 있는 형태로 저장

# 이미지 폴더와 어노테이션 폴더를 반복하여 처리
for image_folder, annotation_folder in zip(image_folders, annotation_folders):
    # 이미지 폴더 내의 파일 목록을 가져옴
    image_files = os.listdir(image_folder)

    # 이미지 파일과 어노테이션 파일을 한 쌍씩 처리
    for image_file in tqdm(image_files):
        # 이미지 파일의 경로
        image_path = os.path.join(image_folder, image_file)

        # 어노테이션 파일의 경로
        annotation_file = os.path.splitext(image_file)[0] + '.json'
        annotation_path = os.path.join(annotation_folder, annotation_file)

        # 어노테이션 파일을 열어 bbox 정보를 추출하여 텍스트 파일로 저장
        with open(annotation_path, 'r') as f:
            annotation_data = json.load(f)

            # 이미지 정보와 어노테이션 정보에서 필요한 값 추출
            image_info = annotation_data['images'][0]
            annotations = annotation_data['annotations']

            # 이미지 정보에서 파일명 추출
            image_filename = image_info['filename']
            image_width = image_info['width']
            image_height = image_info['height']

            # bbox 값이 있는 어노테이션만 처리하여 텍스트 파일로 저장 (bbox 값 가지지 않는 이미지 존재하기 때문)
            annotated_annotations = [annotation for annotation in annotations if 'bbox' in annotation and 'category_id' in annotation]

            if len(annotated_annotations) > 0:
                # 텍스트 파일 경로
                txt_file = os.path.splitext(image_file)[0] + '.txt'
                txt_path = os.path.join(annotation_folder, txt_file) # annotation(bbox) .txt 저장 경로

                with open(txt_path, 'w') as txt_f:
                    for annotation in annotated_annotations:
                        bbox = annotation['bbox']
                        category_id = annotation['category_id']

                        x_center, y_center, norm_width, norm_height = normalize_bbox(bbox, image_width, image_height)
                        class_index = category_id - 381  # class 0부터
                        line = f'{class_index} {x_center} {y_center} {norm_width} {norm_height}\n'
                        txt_f.write(line)
            else:
                # bbox 값이 없는 어노테이션 -> 건너뛰고 처리하지 않음
                continue

" # (각각의 폴더에서) 이미지와 각 이미지에 대한 2D 어노테이션 파일\n# YOLOv8 detection 모델에 적용할 수 있는 형태로 저장\n\n# 이미지 폴더와 어노테이션 폴더를 반복하여 처리\nfor image_folder, annotation_folder in zip(image_folders, annotation_folders):\n    # 이미지 폴더 내의 파일 목록을 가져옴\n    image_files = os.listdir(image_folder)\n\n    # 이미지 파일과 어노테이션 파일을 한 쌍씩 처리\n    for image_file in tqdm(image_files):\n        # 이미지 파일의 경로\n        image_path = os.path.join(image_folder, image_file)\n\n        # 어노테이션 파일의 경로\n        annotation_file = os.path.splitext(image_file)[0] + '.json'\n        annotation_path = os.path.join(annotation_folder, annotation_file)\n\n        # 어노테이션 파일을 열어 bbox 정보를 추출하여 텍스트 파일로 저장\n        with open(annotation_path, 'r') as f:\n            annotation_data = json.load(f)\n\n            # 이미지 정보와 어노테이션 정보에서 필요한 값 추출\n            image_info = annotation_data['images'][0]\n            annotations = annotation_data['annotations']\n\n            # 이미지 정보에서 파일명 추출\n            image_filename = image_info['filename']\n            image

### train

In [7]:
''' image_directory = './train_YOLOv8/'
label_directory = './2D_JSON/'

# 이미지 디렉토리에서 파일 목록 가져오기
image_files = os.listdir(image_directory)

for image_file in image_files:
    # 이미지 파일 이름에서 확장자 제거
    image_name = os.path.splitext(image_file)[0]
    
    # 텍스트 파일 경로 생성
    label_file_path = os.path.join(label_directory, image_name + '.txt')
    
    # 텍스트 파일이 존재하면 복사
    if os.path.isfile(label_file_path):
        shutil.copy(label_file_path, image_directory) '''

" image_directory = './train_YOLOv8/'\nlabel_directory = './2D_JSON/'\n\n# 이미지 디렉토리에서 파일 목록 가져오기\nimage_files = os.listdir(image_directory)\n\nfor image_file in image_files:\n    # 이미지 파일 이름에서 확장자 제거\n    image_name = os.path.splitext(image_file)[0]\n    \n    # 텍스트 파일 경로 생성\n    label_file_path = os.path.join(label_directory, image_name + '.txt')\n    \n    # 텍스트 파일이 존재하면 복사\n    if os.path.isfile(label_file_path):\n        shutil.copy(label_file_path, image_directory) "

### Validation

In [8]:
''' image_directory = './validation_YOLOv8/'
label_directory = './2D_JSON/'

# 이미지 디렉토리에서 파일 목록 가져오기
image_files = os.listdir(image_directory)

for image_file in image_files:
    # 이미지 파일 이름에서 확장자 제거
    image_name = os.path.splitext(image_file)[0]
    
    # 텍스트 파일 경로 생성
    label_file_path = os.path.join(label_directory, image_name + '.txt')
    
    # 텍스트 파일이 존재하면 복사
    if os.path.isfile(label_file_path):
        shutil.copy(label_file_path, image_directory) '''

" image_directory = './validation_YOLOv8/'\nlabel_directory = './2D_JSON/'\n\n# 이미지 디렉토리에서 파일 목록 가져오기\nimage_files = os.listdir(image_directory)\n\nfor image_file in image_files:\n    # 이미지 파일 이름에서 확장자 제거\n    image_name = os.path.splitext(image_file)[0]\n    \n    # 텍스트 파일 경로 생성\n    label_file_path = os.path.join(label_directory, image_name + '.txt')\n    \n    # 텍스트 파일이 존재하면 복사\n    if os.path.isfile(label_file_path):\n        shutil.copy(label_file_path, image_directory) "

### Test

In [9]:
''' image_directory = './test_YOLOv8/'
label_directory = './2D_JSON/'

# 이미지 디렉토리에서 파일 목록 가져오기
image_files = os.listdir(image_directory)

for image_file in image_files:
    # 이미지 파일 이름에서 확장자 제거
    image_name = os.path.splitext(image_file)[0]
    
    # 텍스트 파일 경로 생성
    label_file_path = os.path.join(label_directory, image_name + '.txt')
    
    # 텍스트 파일이 존재하면 복사
    if os.path.isfile(label_file_path):
        shutil.copy(label_file_path, image_directory) '''

" image_directory = './test_YOLOv8/'\nlabel_directory = './2D_JSON/'\n\n# 이미지 디렉토리에서 파일 목록 가져오기\nimage_files = os.listdir(image_directory)\n\nfor image_file in image_files:\n    # 이미지 파일 이름에서 확장자 제거\n    image_name = os.path.splitext(image_file)[0]\n    \n    # 텍스트 파일 경로 생성\n    label_file_path = os.path.join(label_directory, image_name + '.txt')\n    \n    # 텍스트 파일이 존재하면 복사\n    if os.path.isfile(label_file_path):\n        shutil.copy(label_file_path, image_directory) "

# YOLOv8 모델 학습(B-box 인식 모델)

In [10]:
import ultralytics
from ultralytics import YOLO

ultralytics.checks()

Ultralytics YOLOv8.0.105  Python-3.7.16 torch-1.13.1+cpu CPU
Setup complete  (32 CPUs, 127.9 GB RAM, 547.8/930.9 GB disk)


In [ ]:
model = YOLO('yolov8n.pt')

''' model = YOLO("yolov8n.pt", device='gpu') 
model.to('cuda') '''

print(type(model.names), len(model.names))
print(model.names)

In [ ]:
#YOLOv8 모델은 custom.yaml 파일 학습함
model.train(data='./Roadmark_Data.yaml', epochs=50, patience=30, batch=16, imgsz=256)

In [14]:
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 53
{0: '381', 1: '382', 2: '383', 3: '384', 4: '385', 5: '386', 6: '387', 7: '388', 8: '389', 9: '390', 10: '391', 11: '392', 12: '393', 13: '440', 14: '394', 15: '395', 16: '396', 17: '397', 18: '398', 19: '399', 20: '400', 21: '401', 22: '402', 23: '403', 24: '404', 25: '405', 26: '406', 27: '407', 28: '408', 29: '409', 30: '410', 31: '411', 32: '412', 33: '413', 34: '414', 35: '415', 36: '416', 37: '417', 38: '418', 39: '419', 40: '420', 41: '421', 42: '422', 43: '424', 44: '425', 45: '426', 46: '427', 47: '428', 48: '429', 49: '430', 50: '431', 51: '432', 52: '433'}


In [ ]:
results = model.predict(source='./test_YOLOv8/', #테스트 이미지 파일 폴더 경로
                        save=True) #예측 결과~이미지 저장